In [1]:
import sys
import time
import getopt
from blindIssuing_dl_version2 import initialization, tracer_choose_keypair, Issuer, User, verify
import pandas as pd

In [2]:
L = 1024
round = 10
m = b'my msg'

### Issuing Test

In [3]:
def issue_test(L, round, time_recoder=[], flag=1):    
    params = initialization(L)
    traceKeypair = tracer_choose_keypair(params)
    tkey = traceKeypair.yt
    xt = traceKeypair.xt
    total = 0
    for x in range(round):
        start = time.time()
        issuer = Issuer(params,tkey)
        # get the hash 
        issuer.start()

        user = User(params, issuer.IssuerKeypair.y,tkey)    
        user.start()

        zu, xi = user.protocol_one()    
        z1, a, b1, b2 = issuer.protocol_two(zu,xi)    
        e = user.protocol_three(z1, a, b1, b2, m)    
        r, c, s1, s2, d = issuer.protocol_four(e)    
        roi, pi, sigma1, sigma2, delta = user.protocol_five(r, c, s1, s2, d)

        end = time.time()
        runtime = end - start
        if not flag: time_recoder.append(runtime)
        msg = '{func} takes {time} second to complete'
        total = total + runtime
        if flag: print(msg.format(func = '[Credential Issuing]', time = runtime))
    if flag: print("The average time is {time} second for {rounds} tests".format(time=total/round, rounds=round))

In [4]:
issue_test(L, round)

[Credential Issuing] takes 0.006581783294677734 second to complete
[Credential Issuing] takes 0.006831645965576172 second to complete
[Credential Issuing] takes 0.006582736968994141 second to complete
[Credential Issuing] takes 0.006568193435668945 second to complete
[Credential Issuing] takes 0.006573200225830078 second to complete
[Credential Issuing] takes 0.00668025016784668 second to complete
[Credential Issuing] takes 0.006635904312133789 second to complete
[Credential Issuing] takes 0.006600141525268555 second to complete
[Credential Issuing] takes 0.006631135940551758 second to complete
[Credential Issuing] takes 0.006545066833496094 second to complete
The average time is 0.006623005867004395 second for 10 tests


---

### Parameter Test

In [5]:
def parameter_test(L, round, time_recoder=[], flag=1):
    total = 0
    for x in range(round):
        start = time.time()
        params = initialization(L)
        traceKeypair = tracer_choose_keypair(params)
        tkey = traceKeypair.yt
        xt = traceKeypair.xt
        end = time.time()
        runtime = end - start
        if not flag: time_recoder.append(runtime)
        msg = '{func} takes {time} second to complete'
        total = total + runtime
        if flag: print(msg.format(func = '[Parameter Generation]', time = runtime))
    if flag: print("The average time is {time} second for {rounds} tests".format(time=total/round, rounds=round))

In [6]:
parameter_test(L, round)

[Parameter Generation] takes 1.2227978706359863 second to complete
[Parameter Generation] takes 0.15206646919250488 second to complete
[Parameter Generation] takes 1.6951746940612793 second to complete
[Parameter Generation] takes 1.086684226989746 second to complete
[Parameter Generation] takes 1.9909718036651611 second to complete
[Parameter Generation] takes 3.2271177768707275 second to complete
[Parameter Generation] takes 3.7894954681396484 second to complete
[Parameter Generation] takes 3.059931755065918 second to complete
[Parameter Generation] takes 8.676132917404175 second to complete
[Parameter Generation] takes 2.6000592708587646 second to complete
The average time is 2.7500432252883913 second for 10 tests


---

### Verifying Test

In [7]:
def verifying_test(L, round, time_recoder=[], flag=1):
    total = 0
    params = initialization(L)
    traceKeypair = tracer_choose_keypair(params)
    tkey = traceKeypair.yt
    xt = traceKeypair.xt
    
    for x in range(round):
        issuer = Issuer(params, tkey)
        issuer.start()

        user = User(params, issuer.IssuerKeypair.y,tkey)    
        user.start()

        zu, xi = user.protocol_one()    
        z1, a, b1, b2 = issuer.protocol_two(zu,xi)    
        e = user.protocol_three(z1, a, b1, b2, m)    
        r, c, s1, s2, d = issuer.protocol_four(e)    
        roi, pi, sigma1, sigma2, delta = user.protocol_five(r, c, s1, s2, d)
        
        start = time.time()
        verify(roi, pi, sigma1, sigma2, delta, params, m, user.y, user.zeta1,user.zeta2, user.z)
        end = time.time()
        runtime = end - start
        if not flag: time_recoder.append(runtime)
        msg = '{func} takes {time} second to complete'
        total = total + runtime
        if flag: print(msg.format(func = '[Credential Verifying]', time = runtime))
    if flag:print("The average time is {time} second for {rounds} tests".format(time=total/round, rounds=round))

In [8]:
verifying_test(L, round)

[Credential Verifying] takes 0.0030126571655273438 second to complete
[Credential Verifying] takes 0.0031728744506835938 second to complete
[Credential Verifying] takes 0.003055095672607422 second to complete
[Credential Verifying] takes 0.003041505813598633 second to complete
[Credential Verifying] takes 0.0030112266540527344 second to complete
[Credential Verifying] takes 0.003015279769897461 second to complete
[Credential Verifying] takes 0.003070354461669922 second to complete
[Credential Verifying] takes 0.003075122833251953 second to complete
[Credential Verifying] takes 0.003068685531616211 second to complete
[Credential Verifying] takes 0.0030694007873535156 second to complete
The average time is 0.0030592203140258787 second for 10 tests


---

In [16]:
times = 1000
L = 1024
issue_time = []
parameter_time = []
verify_time = []
identify = ['Average Time']
identify.extend(['#'+str(x) for x in range(1,times + 1)])
issue_test(L, times, issue_time, 0)
parameter_test(L, times, parameter_time, 0)
verifying_test(L, times, verify_time, 0)
avg1 = sum(issue_time)/times
avg2 = sum(parameter_time)/times
avg3 = sum(verify_time)/times
issue_time.insert(0, avg1)
parameter_time.insert(0, avg2)
verify_time.insert(0, avg3)
dataframe = pd.DataFrame({'Rounds [TEST DATE: ' + time.strftime('%Y-%m-%d %H:%M',time.localtime(time.time())) + ']':identify, 'Issuing Time':issue_time,'Parameter Time': parameter_time, 'Verifying Time': verify_time})
dataframe.to_csv('./DL_1024_result.csv', index=False)

In [15]:
times = 1000
L = 256
issue_time = []
parameter_time = []
verify_time = []
identify = ['Average Time']
identify.extend(['#'+str(x) for x in range(1,times + 1)])
issue_test(L, times, issue_time, 0)
parameter_test(L, times, parameter_time, 0)
verifying_test(L, times, verify_time, 0)
avg1 = sum(issue_time)/times
avg2 = sum(parameter_time)/times
avg3 = sum(verify_time)/times
issue_time.insert(0, avg1)
parameter_time.insert(0, avg2)
verify_time.insert(0, avg3)
dataframe = pd.DataFrame({'Rounds [TEST DATE: ' + time.strftime('%Y-%m-%d %H:%M',time.localtime(time.time())) + ']':identify, 'Issuing Time':issue_time,'Parameter Time': parameter_time, 'Verifying Time': verify_time})
dataframe.to_csv('./DL_256_result.csv', index=False)